In [ ]:
!git clone https://github.com/google/flatbuffers.git
!cd flatbuffers && cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release && make -j

!cd flatbuffers/python && python3 setup.py install

Cloning into 'flatbuffers'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18457 (delta 6), reused 1 (delta 0), pack-reused 18441
Receiving objects: 100% (18457/18457), 11.10 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (12811/12811), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
-- 

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs

--2020-09-18 07:53:28--  https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26964 (26K) [text/plain]
Saving to: ‘schema.fbs’

schema.fbs          100%[===================>]  26.33K  --.-KB/s    in 0.01s   

2020-09-18 07:53:29 (2.14 MB/s) - ‘schema.fbs’ saved [26964/26964]



In [ ]:
!alias python=python3
!./flatbuffers/flatc --python ./schema.fbs

In [ ]:
!wget https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/hand_landmark.tflite

--2020-09-18 07:53:29--  https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/hand_landmark.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4086864 (3.9M) [application/octet-stream]
Saving to: ‘hand_landmark.tflite’

hand_landmark.tflit 100%[===================>]   3.90M  17.8MB/s    in 0.2s    

2020-09-18 07:53:29 (17.8 MB/s) - ‘hand_landmark.tflite’ saved [4086864/4086864]



In [ ]:
import os
import numpy as np
from collections import OrderedDict
from tflite import Model

In [ ]:
data = open("./hand_landmark.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)
subgraph = model.Subgraphs(0)
subgraph.Name()

b'keras2tflite_handskeleton_handflag_handedness_2020_05_11_v0.tflite.generated'

In [ ]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(tensor)))

  0                     b'input_1' 0  0 [1, 256, 256, 3]
  1               b'conv2d/Kernel' 1  1 [24, 3, 3, 3]
  2                 b'conv2d/Bias' 1  2 [24]
  3                      b'conv2d' 0  0 [1, 128, 128, 24]
  4                       b're_lu' 0  0 [1, 128, 128, 24]
  5     b'depthwise_conv2d/Kernel' 1  3 [1, 5, 5, 24]
  6       b'depthwise_conv2d/Bias' 1  4 [24]
  7            b'depthwise_conv2d' 0  0 [1, 128, 128, 24]
  8             b'conv2d_1/Kernel' 1  5 [24, 1, 1, 24]
  9               b'conv2d_1/Bias' 1  6 [24]
 10                    b'conv2d_1' 0  0 [1, 128, 128, 24]
 11         b'add__xeno_compat__1' 0  0 [1, 128, 128, 24]
 12                  b'activation' 0  0 [1, 128, 128, 24]
 13   b'depthwise_conv2d_1/Kernel' 1  7 [1, 5, 5, 24]
 14     b'depthwise_conv2d_1/Bias' 1  8 [24]
 15          b'depthwise_conv2d_1' 0  0 [1, 128, 128, 24]
 16             b'conv2d_2/Kernel' 1  9 [24, 1, 1, 24]
 17               b'conv2d_2/Bias' 1 10 [24]
 18                    b'conv2d_2' 0  0 

In [ ]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

In [ ]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

189

In [ ]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    print()
    assert(tensor.Type() == 1)  # FLOAT16
    
    W = (model.Buffers(buffer).DataAsNumpy()) # -128.0)/128.0 # .astype(np.float16)
    # print(W.dtype, W.min(), W.max())
    # print(W)
    W = W.view(dtype=np.float16).astype(np.float32)
    print(W[0])
    W = W.reshape(shape)
    return W
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
print(W.shape, b)


-0.35009766

-0.8208008
(24, 3, 3, 3) [-0.8208008  -0.4885254   0.0592041   1.5595703   0.5727539  -1.4599609
 -1.3300781   0.34521484 -1.2519531   0.5986328  -1.3183594   0.55126953
  0.8828125   0.83984375  1.2480469  -1.7783203   0.32104492  0.1385498
  0.49194336  0.14013672  0.09820557 -0.72998047  0.33691406  0.48266602]


In [ ]:
import torch
from handlandmarks import HandLandmarks

In [ ]:
net = HandLandmarks()
net

HandLandmarks(
  (backbone1): Sequential(
    (0): ConstantPad2d(padding=(0, 1, 0, 1), value=0.0)
    (1): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2))
    (2): ReLU(inplace=True)
    (3): ResBlock(
      (f): Sequential(
        (0): ResModule(
          (convs): Sequential(
            (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
            (1): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
        (1): ResModule(
          (convs): Sequential(
            (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
            (1): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
          )
          (act): ReLU(inplace=True)
        )
      )
    )
    (4): ResModule(
      (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (convs): Sequential(
        (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(2, 2), groups=24

In [ ]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 1:
        probable_names.append(tensor.Name().decode("utf-8"))
        
probable_names[:5], len(probable_names)

(['conv2d/Kernel',
  'conv2d/Bias',
  'depthwise_conv2d/Kernel',
  'depthwise_conv2d/Bias',
  'conv2d_1/Kernel'],
 182)

In [ ]:
convert = {}
i = 0
print(len(net.state_dict().items()))
for name, params in net.state_dict().items():
    # print(i)
    # print(name)
    # if i >= len(probable_names):
    #  break
    convert[name] = probable_names[i]
    i += 1

182


In [ ]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:
        if W.shape[0] == 1 and "depthwise" in src:
            W = W.transpose((3, 0, 1, 2))  # depthwise conv # not necessary ughhhh
        else:
          if "transpose" in src:
            W = W.transpose((3, 0, 1, 2)) # transpose
          else:
            W = W.transpose((0, 3, 1, 2))  # regular conv
    
    new_state_dict[dst] = torch.from_numpy(W)



-0.35009766
backbone1.1.weight conv2d/Kernel (24, 3, 3, 3) torch.Size([24, 3, 3, 3])

-0.8208008
backbone1.1.bias conv2d/Bias (24,) torch.Size([24])

-0.065979004
backbone1.3.f.0.convs.0.weight depthwise_conv2d/Kernel (1, 5, 5, 24) torch.Size([24, 1, 5, 5])

0.0
backbone1.3.f.0.convs.0.bias depthwise_conv2d/Bias (24,) torch.Size([24])

-0.43481445
backbone1.3.f.0.convs.1.weight conv2d_1/Kernel (24, 1, 1, 24) torch.Size([24, 24, 1, 1])

-0.35302734
backbone1.3.f.0.convs.1.bias conv2d_1/Bias (24,) torch.Size([24])

-0.079711914
backbone1.3.f.1.convs.0.weight depthwise_conv2d_1/Kernel (1, 5, 5, 24) torch.Size([24, 1, 5, 5])

0.0
backbone1.3.f.1.convs.0.bias depthwise_conv2d_1/Bias (24,) torch.Size([24])

-0.06829834
backbone1.3.f.1.convs.1.weight conv2d_2/Kernel (24, 1, 1, 24) torch.Size([24, 24, 1, 1])

0.83251953
backbone1.3.f.1.convs.1.bias conv2d_2/Bias (24,) torch.Size([24])

0.08618164
backbone1.4.convs.0.weight depthwise_conv2d_2/Kernel (1, 5, 5, 24) torch.Size([24, 1, 5, 5])

0.0

In [ ]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [ ]:
torch.save(net.state_dict(), "HandLandmarks.pth")